#### Данное решение не является решением изначальной задачи. 

Здесь мы хотим узнать затраты на 1 итерацию запланированных ремонтов. Изначальная же задача состоит в том, чтобы для всего временного интервала и всех вагонов отдать даты ремонтов, которые могут происходить раньше запланированных.

In [22]:
import pandas as pd

In [50]:
car = 57463085  # возьмем только один вагон
repares = pd.read_csv('Sibur_2_dataset/Вагоны и плановые ремонты.csv')
repares = repares[repares['car_num'] == car]
repares

,car_num,ct_name,psevdoname,std_kap,std_ppr,std_dep,std_vogi
0,57463085,Газовые,15-78-5,2021-08-08,2018-08-26,2019-08-31,2019-05-19


In [52]:
movements = pd.read_csv('Sibur_2_dataset/Передвижения вагонов по датам.csv')
movements = movements[movements['car_num'] == car]
movements.head()

,car_num,date,station_id
0,57463085,2015-01-01,2
1,57463085,2015-01-25,6
2,57463085,2015-02-18,9
3,57463085,2015-03-14,5
4,57463085,2015-04-07,8


In [51]:
repare_prices = pd.read_csv('Sibur_2_dataset/Плановые ремонты со стоимостями.csv')
repare_prices

,id,repair_name,repair_cost
0,1,Капитальный ремонт,94000000
1,2,Деповской ремонт,19000000
2,3,Плановый предупредительный ремонт,13000000
3,4,Вакуумная очистка и гидроиспытания (ВОиГИ),18000000


In [68]:
repare_prices = {'std_kap': 94000000, 
                 'std_ppr': 19000000, 
                 'std_dep': 13000000, 
                 'std_vogi': 18000000, 
                 'preparation': 3000000}
price_for_km = 1000

In [54]:
distances = pd.read_csv('Sibur_2_dataset/Расстояния от станций до ремонтных депо.csv')
distances.head()

,station_id,name,depo_id,depo_name,distance_to_depo
0,1,Сосновка,1,Депо-Ленинское,10077
1,1,Сосновка,2,Депо-Комсомольское,10436
2,1,Сосновка,3,Депо-Молодежное,10522
3,2,Липовка,1,Депо-Ленинское,2084
4,2,Липовка,2,Депо-Комсомольское,3789


#### Посчитаем стоимость ремонтов, в случае, если делаем их по расписанию. 

Тут нужно сделать замечание, что мы считаем только 1 итерацию ремонтов. Правильнее будет посчитать все итерации: то есть если капитальный ремонт запланирован на 2021-08-08, но мы видим, что вагон эксплуатировался 2019 году, то 2019-08-08 у него должен был быть запланированный капитальный ремонт, т.к. ремонты утилизируются за 2 года

In [73]:
def get_station_from_date(date):
    for index, (car_num, station_date, station_id) in movements.iterrows():
        if station_date < date:
            return station_id
    assert False, 'No stations before this date'

total_repare_price = 0
for repare_type in ['std_kap', 'std_ppr', 'std_dep', 'std_vogi']:
    repare_date = repares[repare_type].values[0]
    station_id = get_station_from_date(repare_date)
    min_distance_to_depo = distances[distances['station_id'] == 2]['distance_to_depo'].min()
    total_repare_price += 2 * min_distance_to_depo * price_for_km + \
            repare_prices[repare_type] + \
            repare_prices['preparation']

total_repare_price

172672000